In [8]:
import os
import requests
import datetime
import json
from pprint import pprint
import time
from tqdm import tqdm


def get_token_id(file_name):
    with open(os.path.join(os.getcwd(), file_name), 'r') as token_file:
        token = token_file.readline().strip()
        id = token_file.readline().strip()
    return [token, id]


def find_max_dpi(dict_photos):
    max_dpi = 0
    for i in range(len(dict_photos)):
        file_dpi = dict_photos[i].get('width') * dict_photos[i].get('height')
        if file_dpi > max_dpi:
            max_dpi = file_dpi
            need_elem = i
    return dict_photos[need_elem].get('url'), dict_photos[need_elem].get('type')


def time_convert(time_unix):
    time_bc = datetime.datetime.fromtimestamp(time_unix)
    str_time = time_bc.strftime('%Y-%m-%d time %H-%M-%S')
    return str_time


class VK_get_request:

    def __init__(self, token_list, version='5.131'):
        self.token = token_list[0]
        self.id = token_list[1]
        self.version = version
        self.start_params = {'access_token': self.token, 'v': self.version}
        self.json, self.export_dict = self._sort_photo_information()

    def _get_photo_information(self):
        url = 'https://api.vk.com/method/photos.get'
        params = {'owner_id': self.id,
                  'album_id': 'profile',
                  'photo_sizes': 1,
                  'extended': 1}
        photo_info = requests.get(url, params={**self.start_params, **params}).json()['response']
        return photo_info['count'], photo_info['items']

    def _get_logs_photo(self):
        photo_count, photo_items = self._get_photo_information()
        result = {}
        for j in range(photo_count):
            likes_count = photo_items[j]['likes']['count']
            url_download, picture_size = find_max_dpi(photo_items[j]['sizes'])
            time_warp = time_convert(photo_items[j]['date'])

            new_value = result.get(likes_count, [])
            new_value.append({'add_name': time_warp,
                              'url_picture': url_download,
                              'size': picture_size})
            result[likes_count] = new_value
        return result

    def _sort_photo_information(self):
        json_list = []
        sorted_dict = {}
        picture_dict = self._get_logs_photo()
        for elem in picture_dict.keys():
            for value in picture_dict[elem]:
                if len(picture_dict[elem]) == 1:
                    file_name = f'{elem}.jpeg'
                else:
                    file_name = f'{elem} {value["add_name"]}.jpeg'
                json_list.append({'file name': file_name, 'size': value["size"]})
                sorted_dict[file_name] = picture_dict[elem][0]['url_picture']
        return json_list, sorted_dict


class Yandex_upload:
    def __init__(self, folder_name, token_list):
        self.token = token_list[0]
        self.url = "https://cloud-api.yandex.net/v1/disk/resources/upload"
        self.headers = {'Authorization': self.token}
        self.folder = self._create_folder(folder_name)

    def _create_folder(self, folder_name):
        url = "https://cloud-api.yandex.net/v1/disk/resources"
        params = {'path': folder_name}
        if requests.get(url, headers=self.headers, params=params).status_code != 200:
            requests.put(url, headers=self.headers, params=params)
            print(f'\nПапка {folder_name} успешно создана в корневом каталоге Яндекс диска\n')
        else:
            print(f'\nПапка {folder_name} уже существует. Файлы с одинаковыми именами не будут скопированы\n')
        return folder_name

    def _in_folder(self, folder_name):
        url = "https://cloud-api.yandex.net/v1/disk/resources"
        params = {'path': folder_name}
        resource = requests.get(url, headers=self.headers, params=params).json()['_embedded']['items']
        in_folder_list = [] 
        for elem in resource:
            in_folder_list.append(elem['name'])
        return in_folder_list

    def create_copy(self, dict_files):
        files_in_folder = self._in_folder(self.folder)
        added_files_num = 0
        for key in tqdm(dict_files.keys()):
            time.sleep(1)
            if key not in files_in_folder:
                params = {'path': f'{self.folder}/{key}',
                          'url': dict_files[key],
                          'overwrite': 'false'}
                requests.post(self.url, headers=self.headers, params=params)
                print(f'Файл {key} успешно добавлен')
                added_files_num += 1
            else:
                print(f'Копирование отменено:Файл {key} уже существует')
        print(f'\nЗапрос завершен, новых файлов добавлено: {added_files_num}')

if __name__ == '__main__':

    tokenVK = 'tokenVK.txt'  # токен и id доступа хранятся в файле
    tokenYandex = 'tokenYandex.txt'  # хранится токен яндекс диска

    my_VK = VK_get_request(get_token_id(tokenVK))
    pprint(my_VK.json)

    my_yandex = Yandex_upload('VK photo copies', get_token_id(tokenYandex))
    my_yandex.create_copy(my_VK.export_dict)

[{'file name': '17.jpeg', 'size': 'r'},
 {'file name': '11.jpeg', 'size': 'y'},
 {'file name': '6.jpeg', 'size': 'y'},
 {'file name': '9 2020-07-06 time 11-53-49.jpeg', 'size': 'y'},
 {'file name': '9 2020-07-06 time 11-54-56.jpeg', 'size': 'w'}]

Папка VK photo copies успешно создана в корневом каталоге Яндекс диска



 20%|██        | 1/5 [00:01<00:07,  1.95s/it]

Файл 17.jpeg успешно добавлен


 40%|████      | 2/5 [00:03<00:05,  1.85s/it]

Файл 11.jpeg успешно добавлен


...

Файл 6.jpeg успешно добавлен


 80%|████████  | 4/5 [00:07<00:01,  1.89s/it]

Файл 9 2020-07-06 time 11-53-49.jpeg успешно добавлен


100%|██████████| 5/5 [00:09<00:00,  1.87s/it]

Файл 9 2020-07-06 time 11-54-56.jpeg успешно добавлен

Запрос завершен, новых файлов добавлено: 5
